In [67]:
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd

In [68]:
df = pd.read_parquet("/Users/samyamaryal/Desktop/Data Competition/DSU-Data-Competition-Spring2025/data/DSU-Dataset.parquet")

## Basic data check

In [69]:
df.columns

Index(['Claim ID', 'Service Date', 'Recieved Date', 'Paid Date', 'Patient ID',
       'Member Age', 'Gender', 'Marital Status', 'Ethnicity', 'LOB',
       'Network Status', 'Claim Category', 'Claim Subcategory', 'Claim Line',
       'Place of Service', 'Provider Type', 'Provider Specialty',
       'ICD10 Code 1', 'ICD10 Code 2', 'ICD10 Code 3', 'ICD10 Code 4',
       'ICD10 Code 5', 'ICD10 Code 6', 'ICD10 Code 7', 'ICD10 Code 8',
       'ICD10 Code 9', 'ICD10 Code 10', 'Service Type', 'Service Code',
       'Modifiers', 'High Cost Claim'],
      dtype='object')

In [70]:
df.dtypes

Claim ID               object
Service Date           object
Recieved Date          object
Paid Date              object
Patient ID             object
Member Age             object
Gender                 object
Marital Status         object
Ethnicity              object
LOB                    object
Network Status         object
Claim Category         object
Claim Subcategory      object
Claim Line             object
Place of Service       object
Provider Type          object
Provider Specialty     object
ICD10 Code 1          float64
ICD10 Code 2          float64
ICD10 Code 3          float64
ICD10 Code 4          float64
ICD10 Code 5          float64
ICD10 Code 6          float64
ICD10 Code 7          float64
ICD10 Code 8          float64
ICD10 Code 9          float64
ICD10 Code 10         float64
Service Type           object
Service Code           object
Modifiers              object
High Cost Claim       float64
dtype: object

In [71]:
df.head()

,Claim ID,Service Date,Recieved Date,Paid Date,Patient ID,Member Age,Gender,Marital Status,Ethnicity,LOB,...,ICD10 Code 5,ICD10 Code 6,ICD10 Code 7,ICD10 Code 8,ICD10 Code 9,ICD10 Code 10,Service Type,Service Code,Modifiers,High Cost Claim
0,8110afaab1020212f58193e233369c1ac352baf280637c...,2022-01-15,2022-01-17,2022-01-25,99f69b741b2784601452afbe7f9083b5ab984b1b7acc64...,10-19 Yrs Old,F,Single,Caucasian/White,SGH,...,NaN,NaN,NaN,NaN,NaN,NaN,PROC,U0005,None,NaN
1,8110afaab1020212f58193e233369c1ac352baf280637c...,2022-01-15,2022-01-17,2022-01-25,99f69b741b2784601452afbe7f9083b5ab984b1b7acc64...,10-19 Yrs Old,F,Single,Caucasian/White,SGH,...,NaN,NaN,NaN,NaN,NaN,NaN,PROC,U0003,None,NaN
2,190430199dfa0b3ed46212060301b3d743d64f958fa505...,2022-01-05,2022-01-06,2022-01-10,79c25d5d6a0cec975ddcd349f1c431eec1ff698295e752...,10-19 Yrs Old,M,Single,Caucasian/White,NPS,...,NaN,NaN,NaN,NaN,NaN,NaN,PROC,73630,"TC,LT",0.0
3,cac4ce65912da45e6350bf9528623f921762a436056721...,2021-12-28,2021-12-30,2022-01-03,2fe692eb90cbcb4a2afa053ac41691f4a93f7acbd8c357...,50-59 Yrs Old,F,Divorced,Caucasian/White,NPS,...,NaN,NaN,NaN,NaN,NaN,NaN,PROC,87426,None,0.0
4,e01f80a521e095d6fe969a0342a74c23d886c0e324d314...,2022-01-05,2022-01-06,2022-01-13,64c05a5a40591a5f00d0868167a22d37515c65581dd511...,1-9 Yrs Old,F,Single,Caucasian/White,SGH,...,NaN,NaN,NaN,NaN,NaN,NaN,PROC,90670,None,0.0


## Change types of columns to datetime

In [72]:
df[['Service Date', 'Recieved Date', 'Paid Date']] = df[['Service Date', 'Recieved Date', 'Paid Date']].apply(pd.to_datetime)

### Compute time difference between recieved date/ service date and paid date/ recieve date

In [73]:
df['recieve delay'] = (df['Recieved Date'] - df['Service Date']).dt.days
df['payment delay'] = (df['Paid Date'] - df['Recieved Date']).dt.days

In [74]:
df['payment delay'].max()

980

In [75]:
df['Month'] = pd.to_datetime(df['Service Date']).dt.month
df['Day of Week'] = pd.to_datetime(df['Service Date']).dt.dayofweek

## Change Modifier to split them - todo

In [76]:
# df['Modifiers'] = df['Modifiers'].apply(lambda x: x.split(",") if x is not None else None)

# max_length_modifier_idx = df['Modifiers'].dropna().apply(len).idxmax()
# df.loc[max_length_modifier_idx]['Modifiers']


In [77]:
df['Modifiers']

0             None
1             None
2            TC,LT
3             None
4             None
            ...   
253681          59
253682       RR,KX
253683    RR,KJ,KX
253684        None
253685        None
Name: Modifiers, Length: 253686, dtype: object

In [78]:
labels = df['High Cost Claim']
cols_to_drop = ['Claim ID', 'Patient ID', 'Service Date', 'Recieved Date', 'Paid Date', 'Modifiers', 'High Cost Claim']
df_short = df.drop(cols_to_drop, axis=1)

In [79]:
labels.fillna(0, inplace=True)

In [80]:
df_short = pd.get_dummies(df_short, columns=df_short.select_dtypes('object').columns)

In [81]:
df_short.rename({'Member Age_< 1 Yrs Old': 'Member Age_lessthan 1 Yrs Old'}, axis=1, inplace=True) #a feature can't have < as a character

In [82]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(df_short, labels)

In [83]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val) 

In [84]:
X_train.shape, X_test.shape

((142698, 5421), (63422, 5421))

In [85]:
model = XGBClassifier(verbosity=2, eval_metric='aucpr')

In [86]:
model.fit(X_train, y_train, eval_set = [(X_val, y_val)]) 

[0]	validation_0-aucpr:0.72657
[1]	validation_0-aucpr:0.77311
[2]	validation_0-aucpr:0.82849
[3]	validation_0-aucpr:0.85554
[4]	validation_0-aucpr:0.87737
[5]	validation_0-aucpr:0.89214
[6]	validation_0-aucpr:0.90103
[7]	validation_0-aucpr:0.90678
[8]	validation_0-aucpr:0.91194
[9]	validation_0-aucpr:0.91603
[10]	validation_0-aucpr:0.92026
[11]	validation_0-aucpr:0.92509
[12]	validation_0-aucpr:0.92928
[13]	validation_0-aucpr:0.93727
[14]	validation_0-aucpr:0.93941
[15]	validation_0-aucpr:0.94249
[16]	validation_0-aucpr:0.94959
[17]	validation_0-aucpr:0.95342
[18]	validation_0-aucpr:0.95627
[19]	validation_0-aucpr:0.96077
[20]	validation_0-aucpr:0.96276
[21]	validation_0-aucpr:0.96428
[22]	validation_0-aucpr:0.96447
[23]	validation_0-aucpr:0.96894
[24]	validation_0-aucpr:0.97096
[25]	validation_0-aucpr:0.97426
[26]	validation_0-aucpr:0.97924
[27]	validation_0-aucpr:0.98179
[28]	validation_0-aucpr:0.98293
[29]	validation_0-aucpr:0.98342
[30]	validation_0-aucpr:0.98438
[31]	validation_0-

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [87]:
model.score(X_test, y_test)

0.9995427454195704

In [88]:
imp = pd.DataFrame(data={"Features ": model.feature_names_in_, "Importances ": model.feature_importances_}) 

In [89]:
y_pred_probs = model.predict_proba(X_test)
y_preds = model.predict(X_test) 

In [90]:
import sklearn 

sklearn.metrics.f1_score(y_test, y_preds) 

0.9960763090244893

In [91]:
y_test.value_counts()

High Cost Claim
0.0    59720
1.0     3702
Name: count, dtype: int64

In [92]:
pd.Series(y_preds).value_counts()

0    59733
1     3689
Name: count, dtype: int64

In [93]:
sklearn.metrics.confusion_matrix(y_test, y_preds) 

array([[59712,     8],
       [   21,  3681]])